In [1]:
from datagen import net_data
from  model import model

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

tf.config.run_functions_eagerly(True)

Events = 25499


In [2]:
# Define model 
Model = model()
Model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
               loss = ['mse','mse'], loss_weights=[1,1])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 169, 358, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 167, 356, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 84, 178, 32)       25632     
_________________________________________________________________
dropout (Dropout)            (None, 84, 178, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 82, 176, 64)       18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 40, 87, 64)        36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 18, 42, 64)        1

In [ ]:
# Load training and validation data. 0 - datagenerator, 1 - numpy array
k = 1
train, valid = net_data(k, 50000, 25000, 64)

# Train model
if k == 0:     
    history = Model.fit(train, epochs=20, steps_per_epoch=len(train), validation_data=valid)
if k == 1: 
    history = Model.fit(train[0], (train[1], train[2]), epochs=15, batch_size=64, 
                    validation_data=(valid[0], (valid[1], valid[2])))

/local/psimkina/.local/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/15
782/782 [==============================] - 215s 275ms/step - loss: 2.8855 - output_1_loss: 0.2492 - output_2_loss: 2.6363 - val_loss: 2.2766 - val_output_1_loss: 0.1627 - val_output_2_loss: 2.1139
Epoch 2/15
782/782 [==============================] - 202s 259ms/step - loss: 2.0117 - output_1_loss: 0.1042 - output_2_loss: 1.9076 - val_loss: 1.7439 - val_output_1_loss: 0.0901 - val_output_2_loss: 1.6538
Epoch 3/15
782/782 [==============================] - 202s 258ms/step - loss: 1.6213 - output_1_loss: 0.0557 - output_2_loss: 1.5656 - val_loss: 1.4860 - val_output_1_loss: 0.0513 - val_output_2_loss: 1.4348
Epoch 4/15
782/782 [==============================] - 204s 261ms/step - loss: 1.3877 - output_1_loss: 0.0354 - output_2_loss: 1.3524 - val_loss: 1.4813 - val_output_1_loss: 0.0427 - val_output_2_loss: 1.4387
Epoch 5/15
782/782 [==============================] - 204s 261ms/step - loss: 1.2053 - output_1_loss: 0.0248 - output_2_loss: 1.1805 - val_loss: 1.2525 - val_output_1_l

In [ ]:
def untransform(y): 
    y[:,:,0]*=360
    y[:,:,1]*=170
    y[:,:,1]-=85
    return y
X, y, ye = np.load('Xtrain.npy'), np.load('ytrain.npy'), np.load('yetrain.npy')
print(X.shape)
y_pr, ye_pr = Model.predict(X)
y_pr = np.floor(untransform(y_pr))
y = np.floor(untransform(y))
X = X.reshape(-1,171,360)
ye_pr[ye_pr<0.05]=np.nan
ye[ye<0.05]=np.nan

In [ ]:
fig, axis = plt.subplots(10, 3, figsize=(25, 100))
p = 0
for n in range(10):
    ax = axis[n,0]
    eb = ax.pcolormesh(np.linspace(-0.5,10.5,num=12), np.linspace(-0.5,10.5,num=12),
                            ye[n][p], cmap = 'viridis', vmax=np.nanmax(ye[n][0]))
    clb = plt.colorbar( eb , ax = ax)
    clb.ax.tick_params(labelsize=15)
    ax.tick_params(labelsize=15)
    for t in range (0,np.shape(ye[n][p])[0]):
        for l in range(0,np.shape(ye[n][p])[1]):
            if not(np.isnan(ye[n,p,t,l])):
                 ax.text(t, l,'{:0.1f}'.format(ye[n,p,t,l]), ha='center', va='center',
                     bbox=dict(boxstyle='round', facecolor='white', edgecolor='0.3'), fontsize=12)
    
    ax= axis[n,1]
    eb = ax.pcolormesh(np.linspace(-0.5,10.5,num=12), np.linspace(-0.5,10.5,num=12),
                        ye_pr[n][p], cmap = 'viridis', vmax=np.nanmax(ye[n][0]))
    clb = plt.colorbar( eb , ax = ax)
    clb.ax.tick_params(labelsize=15)
    ax.tick_params(labelsize=15)
    for t in range (0,np.shape(ye_pr[n][p])[0]):
        for l in range(0,np.shape(ye_pr[n][p])[1]):
            if not(np.isnan(ye_pr[n,p,t,l])):
                 ax.text(t, l,'{:0.1f}'.format(ye_pr[n,p,t,l]), ha='center', va='center',
                     bbox=dict(boxstyle='round', facecolor='white', edgecolor='0.3'), fontsize=12)
#     ax = axis[n,2]
#     if (y[n,p,0]!=-1):
#         phi0 = int(y[n,p,1]+85)
#         eta0 = int(y[n,p,0]-1)
#         ics = X[n, phi0-5:phi0+6, eta0-5:eta0+6]
#         eb = ax.pcolormesh(np.linspace(-0.5,10.5,num=12), np.linspace(-0.5,10.5,num=12),
#                             ics, cmap = 'viridis', vmax=np.nanmax(ye[n][0]))
#         clb = plt.colorbar( eb , ax = ax)
#         clb.ax.tick_params(labelsize=15)
#         ax.tick_params(labelsize=15)
#         for t in range (0,np.shape(ics)[0]):
#             for l in range(0,np.shape(ics)[1]):
#                 if not(np.isnan(ics[t,l])):
#                      ax.text(t, l,'{:0.1f}'.format(ics[t,l]), ha='center', va='center',
#                          bbox=dict(boxstyle='round', facecolor='white', edgecolor='0.3'), fontsize=12)

In [ ]:
X[X==0]=np.nan
fig, axis = plt.subplots(10, figsize=(10, 100))
for n in range(10):
    ax = axis[n]
    eb = ax.pcolormesh(np.linspace(0.5,360.5,num=361), np.linspace(-85.5,85.5,num=172),
                                X[n], cmap = 'terrain')
    plt.colorbar( eb , ax = ax)
    ax.scatter(y[n, :,0], y[n, :,1], color='r')
    ax.scatter(y_pr[n, :,0], y_pr[n, :,1], color='pink')
    ax.set_xlim(0.5, 360.5)